<a href="https://colab.research.google.com/github/salvatoredebenedictis/ColabDocuments/blob/main/Procedura_di_Trasferimento_Label_semiautomatico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Procedura semiautomatica per il trasferimento delle etichette da immagini a fumetti a immagini originali**

Questa procedura prende in input un dataset creato da roboflow (si assume che siano immagini di fumetti)

## ***Import Librerie***

In [ ]:
%pip install -q roboflow
from google.colab.patches import cv2_imshow
from google.colab import files
from roboflow import Roboflow
import cv2
import numpy as np
import shutil
import os

     |████████████████████████████████| 145 kB 5.0 MB/s 
     |████████████████████████████████| 178 kB 44.5 MB/s 
     |████████████████████████████████| 1.1 MB 31.9 MB/s 
     |████████████████████████████████| 67 kB 1.9 MB/s 
     |████████████████████████████████| 54 kB 1.9 MB/s 
     |████████████████████████████████| 138 kB 35.2 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 


## ***1.Caricamento del dataset a Fumetti***

In [ ]:
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

copiare il frammento di codice contenente le coordinate del dataset suggerito da Roboflow nel momento in cui si esporta il dataset

In [ ]:
rf = Roboflow(api_key="N7iUcDJYn60rIDtGcdtk")
project = rf.workspace("salvatore-de-benedictis").project("procedurafumettiadaptivethreshold")
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/datasets/ProceduraFumettiAdaptiveThreshold-1 in yolov5pytorch:: 100%|██████████| 412/412 [00:00<00:00, 1083.70it/s]


creiamo una copia del dataset

In [ ]:
source_dir = r"/content/datasets/"
destination_dir = r"/content/datasetOutput"
shutil.copytree(source_dir, destination_dir)

'/content/datasetOutput'

## ***2.Caricamento delle immagini originali***

Funzione per il caricamento delle immagini originali (non fumetizzate)

In [ ]:
def read_file(filename):
  img = cv2.imread(filename)
  return img

Caricamento delle immagini originali (non fumetizzate). Si assume che le immagini che si stanno caricando siano numerate progressivamente in modo tale da facilitare il trasferimento delle label

In [ ]:
# Se non esiste la cartella delle immagini, creiamola
if not os.path.exists("input_images/" ):
    os.makedirs("input_images/" )

# Caricamento delle immagini
print("Caricamento immagini ...")
uploaded = files.upload()
filename = next(iter(uploaded))
img = read_file(filename)
# Variabile per salvataggio dei nomi delle immagini caricate
filenames = uploaded.keys()
# Per ogni immagine caricata, spostiamola nella cartella delle immagini
for f in filenames:
    shutil.move(f, "input_images/" + f)
print("Caricamento immagini completato.")

Caricamento immagini ...


Saving 1.png to 1.png
Saving 2.png to 2.png
Saving 3.png to 3.png
Saving 4.png to 4.png
Saving 5.png to 5.png
Saving 6.png to 6.png
Saving 7.png to 7.png
Saving 8.png to 8.png
Saving 9.png to 9.png
Saving 10.png to 10.png
Saving 11.png to 11.png
Saving 12.png to 12.png
Saving 13.png to 13.png
Saving 14.png to 14.png
Saving 15.png to 15.png
Saving 16.png to 16.png
Saving 17.png to 17.png
Saving 18.png to 18.png
Saving 19.png to 19.png
Saving 20.png to 20.png
Saving 21.png to 21.png
Saving 22.png to 22.png
Saving 23.png to 23.png
Saving 24.png to 24.png
Saving 25.png to 25.png
Saving 26.png to 26.png
Saving 27.png to 27.png
Saving 28.png to 28.png
Saving 29.png to 29.png
Saving 30.png to 30.png
Saving 31.png to 31.png
Saving 32.png to 32.png
Saving 33.png to 33.png
Saving 34.png to 34.png
Saving 35.png to 35.png
Saving 36.png to 36.png
Saving 37.png to 37.png
Saving 38.png to 38.png
Saving 39.png to 39.png
Saving 40.png to 40.png
Saving 41.png to 41.png
Saving 42.png to 42.png
Saving 43.

creo una copia backup delle immagini non a fumetti

In [ ]:
source_dir = r"/content/input_images"
destination_dir = r"/content/original_images"
shutil.copytree(source_dir, destination_dir)

'/content/original_images'

## ***3.Trasferimento delle etichette***

Inizializzo i vari path:

inputPath fanno riferimento al dataset dei volti a fumetti

outputPath fanno riferimento al dataset che conterrà le label delle immagini a fumetti ma le immagini originali non a fumetti

OriginalImagesPath conterrà le immagini non a fumetti che devono essere rinominate e sostituite alle immagini a fumetti

In [ ]:
datasetName = os.listdir("/content/datasets/")
datasetNameOut = os.listdir("/content/datasetOutput/")

inputPathImagesTest = ("/content/datasets/"+datasetName[0]+"/test/images")
inputPathImagesTrain = ("/content/datasets/"+datasetName[0]+"/train/images")
inputPathImagesValid = ("/content/datasets/"+datasetName[0]+"/valid/images")

outputPathImagesTest = ("/content/datasetOutput/"+datasetNameOut[0]+"/test/images")
outputPathImagesTrain = ("/content/datasetOutput/"+datasetNameOut[0]+"/train/images")
outputPathImagesValid = ("/content/datasetOutput/"+datasetNameOut[0]+"/valid/images")

originalImagesPath = ("/content/input_images")

AutomaticRename è la funzione che permette di rinominare le immagini non a fumetti con lo stesso nome delle immagini a fumetti.

inputPath: path sorgente contenente le immagini a fumetti

outputPath: path di destinazione contenente le immagini non a fumetti da rinominare

isInformativeDrawings: flag per stabilire se la tecnica utilizzata per rendere a fumetti l'immagine è "informativeDrawings" con 1 o "AdaptiveThreshold" con 0

In [ ]:
def automaticRename(inputPath, outputPath, isInformativeDrawings):

  for images in os.listdir(inputPath):
    #NomeFile generato da roboflow: nomefilecaricato.rf.stringadi32caratteri.jpg
    stringaNomeFile = images
    #Prendo le ultime 37 cifre contenente codice esadecimale aggiunto da Roboflow + . + jpg
    length = len(stringaNomeFile)
    esaCode = stringaNomeFile[length - 36 :]
    nomefile = stringaNomeFile[: length-36]
    if(isInformativeDrawings == 1):
      numerofile = nomefile[:len(nomefile)-12] #8 se adaptive threshold 12 se è informative drawings
    else:
      numerofile = nomefile[:len(nomefile)-8] #8 se adaptive threshold 12 se è informative drawings
    
    #print("Nome intero: "+stringaNomeFile)
    #print("Codice Esadecimale: "+esaCode)
    #print("Nome File senza codice Esadecimale: "+nomefile)
    #print("Nome File originale: "+numerofile)
    for filedaRinominare in os.listdir(outputPath):
      print(numerofile)
      print(filedaRinominare)
      if (numerofile+'.png') == filedaRinominare :
        if(isInformativeDrawings == 1): 
          os.rename(outputPath + '//' + filedaRinominare, outputPath + '//' + numerofile+ '_out_png.rf.'+ esaCode)
        else:
          os.rename(outputPath + '//' + filedaRinominare, outputPath + '//' + numerofile+ '_png.rf.'+ esaCode)
        

  print("Done!")

automaticReplace è una funzione che permette di trasferire le immagini con lo stesso nome da una cartella sorgente(input)  ad una cartella di destinazione (output)

In [ ]:
def automaticReplace(input, output):
  for inputFile in os.listdir(input):
    for outFile in os.listdir(output):
      #print("Input = "+inputFile)
      #print("Output = "+outFile)
      if(inputFile == outFile):
        shutil.move(os.path.join(input, inputFile), os.path.join(output, outFile))
        break

Chiamiamo le due funzioni per le tre cartelle Train, Test, Valid

In [ ]:
automaticRename(inputPathImagesTest,originalImagesPath,0)
automaticReplace(originalImagesPath,outputPathImagesTest)

automaticRename(inputPathImagesTrain,originalImagesPath,0)
automaticReplace(originalImagesPath,outputPathImagesTrain)

automaticRename(inputPathImagesValid,originalImagesPath,0)
automaticReplace(originalImagesPath,outputPathImagesValid)

Done!
Done!
Done!


## ***4.Generazione e Salvataggio del nuovo Dataset***

In [ ]:
#genero lo zip della cartella da scaricare
!zip -r /datasetOutput.zip  /content/datasetOutput/

updating: content/datasetOutput/ (stored 0%)
updating: content/datasetOutput/ProceduraFumettiAdaptiveThreshold-1/ (stored 0%)
updating: content/datasetOutput/ProceduraFumettiAdaptiveThreshold-1/train/ (stored 0%)
updating: content/datasetOutput/ProceduraFumettiAdaptiveThreshold-1/train/images/ (stored 0%)
updating: content/datasetOutput/ProceduraFumettiAdaptiveThreshold-1/train/images/126_png.rf.d9dac4489a7d1daa4168fba6366fa598.jpg (deflated 2%)
updating: content/datasetOutput/ProceduraFumettiAdaptiveThreshold-1/train/images/35_png.rf.9629dcb705c0dd01fc427a31c745e1a5.jpg (deflated 2%)
updating: content/datasetOutput/ProceduraFumettiAdaptiveThreshold-1/train/images/104_png.rf.af82638981ce8eaafa6a170997da9644.jpg (deflated 22%)
updating: content/datasetOutput/ProceduraFumettiAdaptiveThreshold-1/train/images/158_png.rf.5f5f29efe6eba444a0daf6f3aa4cc7a7.jpg (deflated 1%)
updating: content/datasetOutput/ProceduraFumettiAdaptiveThreshold-1/train/images/174_png.rf.7b5c051642f936f2534b1e1731d1a

In [ ]:
#scarico la cartella
files.download("/datasetOutput.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>